In [36]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [37]:
import pyomo.environ as pe
import pyomo.opt as po


In [38]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)                                   # set of all factories
model.T = pe.Set(initialize = transshipment_points ,ordered = False)                        # set of all transshipment points
model.C = pe.Set(initialize = customers ,ordered = False)                                   # set of all customers    
model.M = pe.Set(initialize = modes ,ordered = False)                                       # set of all modes
model.P = pe.Set(initialize = periods, ordered = False)                                     # set of all periods

# Decision Variables
# NOTE je kan ook lower en upper bounds geven aan de waarden
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # quantity transported directly from factory to customer                       
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # quantity transported from transshipment point to customer                 
model.q_bl = pe.Var(model.F, block_trans, model.P, domain = pe.NonNegativeReals)            # quantity transported by mode block train                
model.q_s_w = pe.Var(model.F, single_trans, model.P, domain = pe.NonNegativeReals)          # quantity transported by mode single wagon train            
model.q_ss = pe.Var(model.F, shortsea_trans, model.P, domain = pe.NonNegativeReals)         # quantity transported by mode shortsea shipping           
model.q_ba = pe.Var(model.F, barge_trans, model.P, domain = pe.NonNegativeReals)            # quantity transported by mode barge                    
model.td = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route from factory to customer     
model.tid = pe.Var(model.T, model.C, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route from transshipment point to customer   

In [39]:
# Objective function
objExpr = sum(sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C)      \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans)                 \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans)              \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans)                 \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] for i in model.F for j in barge_trans)                       \
        + sum(dist_point_cust[i, j] * transfer_cost * model.y[i,j,p] for i in model.T for j in model.C)         \
        + sum(handling_costs[i] * model.y[i,j,p] for i in model.T for j in model.C)                             \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C)                             \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C) for p in model.P)          \

model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [40]:
# Contraints

# for each period, for each combination of factory and customer, the number of direct trucks >= the number of direct tonnes / truck capacity
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

# for each period, for each combination of factory and customer, the number of indirect trucks >= the number of indirect tonnes / truck capacity
model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)

# for each period, for each factory, the total number of tonnes transported from the factory should not exceed its production capacity
model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
        model.prodCap.add(expression)

# for each period, for each customer, the demands should be met
model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) >= demands[p][j]
        model.meetDemand.add(expression)

# for each period, for each transshipment point, the number of tonnes coming in should equal the number of tonnes going out
model.flowBalance = pe.ConstraintList()
for p in model.P:
    total_quantity_trans = {"Robert Schmidtz": 0, "Wiechers": 0, "Haeger und Schmidt": 0, "Neska/UCT Neuss": 0,
			"WTA": 0, "Siefert spedition": 0, "Rhenus logistics": 0}
    for j in block_trans:
        expression_bl = sum(model.q_bl[i,j,p] for i in model.F)
        expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F) 
        total_train = expression_bl + expression_s_w
        total_quantity_trans[j] += total_train
    for j in shortsea_trans:
        expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ss
    for j in barge_trans:
        expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ba
    for j in model.T: 
        expression = total_quantity_trans[j] == sum(model.y[j,k,p] for k in model.C)
        model.flowBalance.add(expression)



In [41]:
# Solve with use of gurobi
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-21
Using license file /Users/tessavanliempt/gurobi.lic
Read LP format model from file /var/folders/qp/3z7wqsnn5xn47x6dpmtpq_vh0000gp/T/tmp33i82ic8.pyomo.lp
Reading time = 0.16 seconds
x1871: 981 rows, 1871 columns, 3741 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 981 rows, 1871 columns and 3741 nonzeros
Model fingerprint: 0x6b75379d
Variable types: 1071 continuous, 800 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 4089631.2500
Presolve removed 883 rows and 1690 columns
Presolve time: 0.57s
Presolved: 98 rows, 181 columns, 362 nonzeros
Found heuristic

In [45]:
# Print statements all decision variables and objective value
print("Objective value = "+ str(pe.value(model.obj)))
print("Direct Tonnes:\n")
for p in model.P:
    for i in model.F:
        for j in model.C:
            if(pe.value(model.x[i,j,p]) > 0):
                print(str(model.x[i,j,p]) + " = " + str(pe.value(model.x[i,j,p])))

print("\Indirect Tonnes:\n")
for p in model.P:
    for i in model.T:
        for j in model.C:
            if(pe.value(model.y[i,j,p]) > 0):
                print(str(model.y[i,j,p]) + " = " + str(pe.value(model.y[i,j,p])))

print("\Quantity by block train:\n")
for p in model.P:
    for i in model.F:
        for j in block_trans:
            if(pe.value(model.q_bl[i,j,p]) > 0):
                print(str(model.q_bl[i,j,p]) + " = " + str(pe.value(model.q_bl[i,j,p])))

print("\Quantity by single wagon train:\n")
for p in model.P:
    for i in model.F:
        for j in single_trans:
            if(pe.value(model.q_s_w[i,j,p]) > 0):
                print(str(model.q_s_w[i,j,p]) + " = " + str(pe.value(model.q_s_w[i,j,p])))

print("\Quantity by shortsea shipping:\n")
for p in model.P:
    for i in model.F:
        for j in shortsea_trans:
            if(pe.value(model.q_ss[i,j,p]) > 0):
                print(str(model.q_ss[i,j,p]) + " = " + str(pe.value(model.q_ss[i,j,p])))

print("\Quantity by barge:\n")
for p in model.P:
    for i in model.F:
        for j in barge_trans:
            if(pe.value(model.q_ba[i,j,p]) > 0):
                print(str(model.q_ba[i,j,p]) + " = " + str(pe.value(model.q_ba[i,j,p])))

print("\ Number of trucks from factory to customer:\n")
for p in model.P:
    for i in model.F:
        for j in model.C:
            if(pe.value(model.td[i,j,p]) > 0):
                print(str(model.td[i,j,p]) + " = " + str(pe.value(model.td[i,j,p])))

print("\ Number of trucks from transshipment point to customer:\n")
for p in model.P:
    for i in model.T:
        for j in model.C:
            if(pe.value(model.tid[i,j,p]) > 0):
                print(str(model.tid[i,j,p]) + " = " + str(pe.value(model.tid[i,j,p])))

Objective value = 334238.50919999974
Direct Tonnes:

\Indirect Tonnes:

y[Rhenus logistics,Boenen,1] = 86.5
y[Rhenus logistics,Bochum,1] = 55.2
y[Rhenus logistics,Dortmund,1] = 50.7
y[Rhenus logistics,Hagen,1] = 273.9
y[Siefert spedition,Gelsenkirchen,1] = 69.8
y[Siefert spedition,Schwerte,1] = 12.4
y[Wiechers,Iserlohn,1] = 201.4
y[Neska/UCT Neuss,Neuss,1] = 230.7
y[Rhenus logistics,Boenen,2] = 86.2
y[Rhenus logistics,Bochum,2] = 54.6
y[Rhenus logistics,Dortmund,2] = 54.2
y[Rhenus logistics,Hagen,2] = 273.9
y[Siefert spedition,Gelsenkirchen,2] = 68.6
y[Siefert spedition,Schwerte,2] = 12.4
y[Wiechers,Iserlohn,2] = 200.2
y[Neska/UCT Neuss,Neuss,2] = 229.8
y[Rhenus logistics,Boenen,3] = 86.0
y[Rhenus logistics,Bochum,3] = 54.0
y[Rhenus logistics,Dortmund,3] = 57.7
y[Rhenus logistics,Hagen,3] = 273.9
y[Siefert spedition,Gelsenkirchen,3] = 67.3
y[Siefert spedition,Schwerte,3] = 12.5
y[Wiechers,Iserlohn,3] = 199.1
y[Neska/UCT Neuss,Neuss,3] = 228.8
y[Rhenus logistics,Boenen,4] = 85.7
y[Rhenu